In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#!pip install tifffile imagecodecs

In [ ]:
# ==== Cell 2: Imports ====
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import timm
from torchvision import transforms
from tifffile import imread
from PIL import Image
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import functional as TF

In [ ]:
# ==== Cell 3: Dataset Class (using tifffile instead of rasterio) ====
class LSTDataset(Dataset):
    def __init__(self, df, patches_dir, weather_cols):
        self.df = df.reset_index(drop=True)
        self.patches_dir = patches_dir
        self.weather_cols = weather_cols
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.patches_dir, row["filename"])
        # Load full TIFF stack (bands, H, W)
        arr = imread(path)  # shape: (bands, H, W)
        # Ensure arr has shape [bands, H, W]
        if arr.ndim == 2:
            # Single band: expand dims
            arr = np.expand_dims(arr, 0)
        # Bands 1-3 as RGB, band 0 as target
        img_np = arr[[1,2,3], :, :].transpose(1,2,0).astype(np.uint8)
        img = self.transform(img_np)
        target = torch.tensor(arr[0, :, :], dtype=torch.float32).unsqueeze(0)
        weather = torch.tensor(row[self.weather_cols].values.astype(np.float32))
        return img, weather, target


In [ ]:
# ==== Cell 4: Data Preparation ====
df = pd.read_csv("/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv")
weather_cols = ["air_temp_C", "dew_point_C", "relative_humidity_percent", "wind_speed_m_s", "precipitation_in"]
df.dropna(subset=weather_cols + ["filename"], inplace=True)
patches_dir = "/content/drive/MyDrive/PatchedOutput_Cleaned"
dataset = LSTDataset(df, patches_dir, weather_cols)


In [ ]:
# ==== Cell 5: Data Splitting & DataLoaders ====
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0, pin_memory=False)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0, pin_memory=False)


In [ ]:
# ==== Cell 6: UNet Model Definition ====
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)
    def forward(self, x):
        b, C, H, W = x.size()
        y = x.view(b, C, -1).mean(-1)
        y = F.relu(self.fc1(y))
        gamma_beta = torch.sigmoid(self.fc2(y)).view(b, C, 1, 1)
        return x * gamma_beta

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, use_se=True):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch,  out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
        self.use_se = use_se
        if use_se:
            self.se = SEBlock(out_ch)
    def forward(self, x):
        out = self.conv(x)
        return self.se(out) if self.use_se else out

class UNetWithMeteo(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, base_c=64, use_se=True, meteo_dim=5):
        super().__init__()
        feats = [base_c, base_c*2, base_c*4, base_c*8, base_c*16, base_c*32]
        self.enc_blocks = nn.ModuleList()
        self.pools = nn.ModuleList()
        prev = in_channels
        for c in feats:
            self.enc_blocks.append(ConvBlock(prev, c, use_se))
            self.pools.append(nn.MaxPool2d(2))
            prev = c
        bottleneck_ch = feats[-1]
        self.film = nn.Sequential(
            nn.Linear(meteo_dim, bottleneck_ch // 2),
            nn.ReLU(inplace=True),
            nn.Linear(bottleneck_ch // 2, bottleneck_ch * 2),
        )
        self.up_convs = nn.ModuleList()
        self.dec_blocks = nn.ModuleList()
        for i in range(len(feats)-1, 0, -1):
            in_ch, out_ch = feats[i], feats[i-1]
            self.up_convs.append(nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2))
            self.dec_blocks.append(ConvBlock(out_ch*2, out_ch, use_se))
        self.final = nn.Conv2d(feats[0], out_channels, kernel_size=1)
    def forward(self, img, meteo):
        skips, x = [], img
        for enc, pool in zip(self.enc_blocks, self.pools):
            x = enc(x)
            skips.append(x)
            x = pool(x)
        gamma, beta = self.film(meteo).chunk(2, dim=1)
        gamma, beta = gamma[..., None, None], beta[..., None, None]
        x = gamma * x + beta
        for up, dec, skip in zip(self.up_convs, self.dec_blocks, reversed(skips[:-1])):
            x = up(x)
            if skip.shape[2:] != x.shape[2:]:
                skip = TF.center_crop(skip, (x.size(2), x.size(3)))
            x = dec(torch.cat([x, skip], dim=1))
        return self.final(x)

In [ ]:
# ==== Cell 7: Device Check & Optimizer ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = UNetWithMeteo(in_channels=3, meteo_dim=len(weather_cols)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.SmoothL1Loss()


Using device: cuda


In [ ]:
# ==== Cell 8: Training Loop ====
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    # — Training —
    model.train()
    train_losses = []
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} Train", leave=False)
    for imgs, weather, tgt in train_bar:
        imgs, weather, tgt = imgs.to(device), weather.to(device), tgt.to(device)

        optimizer.zero_grad()
        pred = model(imgs, weather)
        # ─── Match spatial dims ──────────────────────────────────────────
        if pred.shape[2:] != tgt.shape[2:]:
            pred = F.interpolate(
                pred,
                size=tgt.shape[2:],
                mode='bilinear',
                align_corners=False
            )
        # ─────────────────────────────────────────────────────────────────

        loss = criterion(pred, tgt)
        loss.backward()
        optimizer.step()

        batch_loss = loss.item()
        train_losses.append(batch_loss)
        avg_train_loss = np.mean(train_losses)

        train_bar.set_postfix(
            batch_loss=f"{batch_loss:.4f}",
            avg_loss=f"{avg_train_loss:.4f}"
        )

    # — Validation —
    model.eval()
    val_losses = []
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1} Val", leave=False)
    with torch.no_grad():
        for imgs, weather, tgt in val_bar:
            imgs, weather, tgt = imgs.to(device), weather.to(device), tgt.to(device)
            val_pred = model(imgs, weather)

            if val_pred.shape[2:] != tgt.shape[2:]:
                val_pred = F.interpolate(
                    val_pred,
                    size=tgt.shape[2:],
                    mode='bilinear',
                    align_corners=False
                )

            batch_val_loss = criterion(val_pred, tgt).item()
            val_losses.append(batch_val_loss)
            avg_val_loss = np.mean(val_losses)

            val_bar.set_postfix(
                batch_val_loss=f"{batch_val_loss:.4f}",
                avg_val_loss=f"{avg_val_loss:.4f}"
            )

    # — Epoch Summary —
    avg_train = np.mean(train_losses)
    avg_val   = np.mean(val_losses)
    print(f"Epoch {epoch+1}: Train Loss {avg_train:.4f}, Val Loss {avg_val:.4f}")

    # — Save Best Model —
    if avg_val < best_val_loss:
        torch.save(model.state_dict(), "/content/best_unet.pth")
        best_val_loss = avg_val
        print("Best model updated.")


Epoch 1: Train Loss 1.6802, Val Loss 1.9324
Best model updated.


Epoch 2: Train Loss 1.1746, Val Loss 1.3989
Best model updated.


Epoch 3: Train Loss 0.9185, Val Loss 2.0099


Epoch 4: Train Loss 0.7456, Val Loss 4.6168


Epoch 5: Train Loss 0.6521, Val Loss 1.5908


Epoch 6: Train Loss 0.5269, Val Loss 1.0584
Best model updated.


Epoch 7: Train Loss 0.4733, Val Loss 1.9114


Epoch 8: Train Loss 0.4124, Val Loss 2.7224


Epoch 9: Train Loss 0.3724, Val Loss 1.3540


Epoch 10: Train Loss 0.3314, Val Loss 2.1572


In [ ]:
# In Colab / Jupyter, prefix with !
!pip install segmentation-models-pytorch timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [ ]:
# Cell 1: Imports & Drive mount
import os
import sys
import subprocess
from tqdm.auto import tqdm

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from torchvision import transforms
from tifffile import imread
from PIL import Image

# Auto-install segmentation_models_pytorch if missing
try:
    import segmentation_models_pytorch as smp
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install",
                           "segmentation-models-pytorch", "timm"])
    import segmentation_models_pytorch as smp

# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cell 2: LSTDataset definition
class LSTDataset(Dataset):
    def __init__(self, df, patches_dir, weather_cols):
        self.df = df.reset_index(drop=True)
        self.patches_dir = patches_dir
        self.weather_cols = weather_cols
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406],
                                 std=[0.229,0.224,0.225]),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        path = os.path.join(self.patches_dir, row["filename"])
        arr  = imread(path)           # (bands, H, W)
        if arr.ndim == 2:
            arr = arr[np.newaxis,...]

        # Bands 1-3 → RGB input; band 0 → continuous LST target
        img_np = arr[[1,2,3]].transpose(1,2,0).astype(np.uint8)
        img    = self.transform(img_np)

        mask   = torch.tensor(arr[0], dtype=torch.float32).unsqueeze(0)
        weather= torch.tensor(row[self.weather_cols].values.astype(np.float32))
        return img, weather, mask



In [ ]:
# Cell 3: Load metadata & create DataLoaders
csv_path   = "/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv"
patches_dir= "/content/drive/MyDrive/PatchedOutput_Cleaned"
weather_cols = [
    "air_temp_C","dew_point_C","relative_humidity_percent",
    "wind_speed_m_s","precipitation_in"
]

df = pd.read_csv(csv_path)
df.dropna(subset=weather_cols + ["filename"], inplace=True)

# Full dataset + 80/20 split
full_ds = LSTDataset(df, patches_dir, weather_cols)
n_train = int(0.8 * len(full_ds))
n_val   = len(full_ds) - n_train
train_ds, val_ds = random_split(full_ds, [n_train, n_val])

BATCH_SIZE = 8
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4, pin_memory=True)



In [ ]:
# Cell 4: Model, L1 loss, optimizer, scheduler
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
).to(DEVICE)

criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scaler    = torch.cuda.amp.GradScaler()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=3, verbose=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-4-946346e4af20>:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler    = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Cell 5: train/validate functions
def train_one_epoch(loader, model, optimizer, device):
    model.train()
    total_loss = 0.0
    for imgs, weather, masks in tqdm(loader, desc="Train", leave=False):
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            preds = model(imgs)
            if masks.shape[2:] != preds.shape[2:]:
                masks = F.interpolate(masks,
                                      size=preds.shape[2:],
                                      mode="nearest")
            loss = criterion(preds, masks)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)


def validate(loader, model, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for imgs, weather, masks in tqdm(loader, desc="Valid", leave=False):
            imgs, masks = imgs.to(device), masks.to(device)
            preds = model(imgs)
            if masks.shape[2:] != preds.shape[2:]:
                masks = F.interpolate(masks,
                                      size=preds.shape[2:],
                                      mode="nearest")
            loss = criterion(preds, masks)
            total_loss += loss.item() * imgs.size(0)
    return total_loss / len(loader.dataset)


In [ ]:
try:
    import imagecodecs
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "imagecodecs"])
    import imagecodecs


In [ ]:
# Cell 6: Main training loop + batch-wise loss display
from tqdm.auto import tqdm
import torch.nn.functional as F

NUM_EPOCHS = 10
best_val   = float('inf')

for epoch in range(1, NUM_EPOCHS+1):
    print(f"\n=== Epoch {epoch}/{NUM_EPOCHS} ===")

    # — Training Phase with per-batch loss —
    model.train()
    train_losses = []
    train_bar = tqdm(
        train_loader,
        desc="Train",
        unit="batch",
        leave=False
    )
    for imgs, weather, masks in train_bar:
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            preds = model(imgs)
            if masks.shape[2:] != preds.shape[2:]:
                masks = F.interpolate(masks, size=preds.shape[2:], mode="nearest")
            loss = criterion(preds, masks)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_losses.append(loss.item())
        train_bar.set_postfix(L1=f"{loss.item():.4f}")

    avg_train = sum(train_losses) / len(train_losses)

    # — Validation Phase with per-batch loss —
    model.eval()
    val_losses = []
    valid_bar = tqdm(
        val_loader,
        desc="Valid",
        unit="batch",
        leave=False
    )
    with torch.no_grad():
        for imgs, weather, masks in valid_bar:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            preds = model(imgs)
            if masks.shape[2:] != preds.shape[2:]:
                masks = F.interpolate(masks, size=preds.shape[2:], mode="nearest")
            loss = criterion(preds, masks)

            val_losses.append(loss.item())
            valid_bar.set_postfix(L1=f"{loss.item():.4f}")

    avg_val = sum(val_losses) / len(val_losses)

    # — Epoch Summary & Checkpoint —
    print(f"Epoch {epoch}/{NUM_EPOCHS} → "
          f"Avg Train L1: {avg_train:.4f} | Avg Val L1: {avg_val:.4f}")

    scheduler.step(avg_val)
    if avg_val < best_val:
        best_val = avg_val
        torch.save(model.state_dict(),
                   "/content/drive/MyDrive/best_unet_resnet34_l1.pth")
        print("🔖 Saved new best model!")

print("✅ Training complete.")



=== Epoch 1/10 ===


Train:   0%|          | 0/1429 [00:00<?, ?batch/s]

<ipython-input-6-4b510d916305>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Valid:   0%|          | 0/358 [00:00<?, ?batch/s]

Epoch 1/10 → Avg Train L1: 2.1946 | Avg Val L1: 1.9213
🔖 Saved new best model!

=== Epoch 2/10 ===


...

Train:   0%|          | 0/1429 [00:00<?, ?batch/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c0f3414ac00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^Exception ignored in: ^<function _MultiProcessingDataLoaderIter.__del__ at 0x7c0f3414ac00>^
^Traceback (most recent call last):
^  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
^^    ^self._shutdown_workers()^
^  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
^^    ^if w.is_alive():^
^ 

Valid:   0%|          | 0/358 [00:00<?, ?batch/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c0f3414ac00><function _MultiProcessingDataLoaderIter.__del__ at 0x7c0f3414ac00>

Traceback (most recent call last):
Exception ignored in: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7c0f3414ac00>  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    
    self._shutdown_workers()self._shutdown_workers()Traceback (most recent call last):


  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
            if w.is_alive():Exce

Epoch 10/10 → Avg Train L1: 0.4640 | Avg Val L1: 0.7998
✅ Training complete.


Focal-Tversky Loss

Kept num_workers=4 and persistent_workers=True plus a small prefetch_factor for speed

A ReduceLROnPlateau cuts the LR by half whenever validation loss stalls, squeezing more out of each run.

In [ ]:
# Cell 1a: Force fork start-method (must come before any DataLoader is created)
import multiprocessing as mp
mp.set_start_method("fork", force=True)


In [ ]:
# Cell 1b: Install dependencies
!pip install segmentation-models-pytorch timm imagecodecs tifffile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-

 Focal Tversky Loss

In [ ]:
# Cell 1: Force fork start method (MUST be very first!)
import multiprocessing as mp
mp.set_start_method('fork', force=True)


In [ ]:
# Cell 2: Install dependencies
!pip install segmentation-models-pytorch timm imagecodecs tifffile


In [ ]:
# Cell 3: Imports & Drive mount
import os
import pandas as pd
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from tifffile import imread
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import torch.nn.functional as F

drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# Cell 4: Dataset + DataLoaders with multi‐worker and safe I/O
def get_dataset_and_loaders(csv_path, patches_dir, weather_cols, batch_size=8):
    df = pd.read_csv(csv_path)
    df.dropna(subset=weather_cols + ['filename'], inplace=True)

    class LSTDataset(Dataset):
        def __init__(self, df, patches_dir, weather_cols):
            self.df = df.reset_index(drop=True)
            self.patches_dir = patches_dir
            self.weather_cols = weather_cols
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224,224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ])
        def __len__(self):
            return len(self.df)
        def __getitem__(self, idx):
            row  = self.df.iloc[idx]
            path = os.path.join(self.patches_dir, row['filename'])
            try:
                arr = imread(path)             # may throw
                if arr.ndim == 2:
                    arr = np.expand_dims(arr,0)
                img_np = arr[[1,2,3]].transpose(1,2,0).astype(np.uint8)
                img    = self.transform(img_np)
                target = torch.clamp(
                    torch.tensor(arr[0],dtype=torch.float32).unsqueeze(0),
                    0.0,1.0
                )
            except Exception as e:
                # fallback to zeros if any I/O error
                print(f"⚠️  Read error {path}: {e}")
                img    = torch.zeros(3,224,224)
                target = torch.zeros(1,224,224)
            weather = torch.tensor(
                row[self.weather_cols].values.astype(np.float32)
            )
            return img, weather, target

    full_ds = LSTDataset(df, patches_dir, weather_cols)
    n_train = int(0.8 * len(full_ds))
    n_val   = len(full_ds) - n_train
    train_ds, val_ds = random_split(full_ds, [n_train, n_val])

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=4, persistent_workers=True,
        pin_memory=True, prefetch_factor=2
    )
    val_loader = DataLoader(
        val_ds, batch_size=batch_size, shuffle=False,
        num_workers=4, persistent_workers=True,
        pin_memory=True, prefetch_factor=2
    )
    return train_loader, val_loader

csv_path    = '/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv'
patches_dir = '/content/drive/MyDrive/PatchedOutput_Cleaned'
weather_cols= ['air_temp_C','dew_point_C',
               'relative_humidity_percent',
               'wind_speed_m_s','precipitation_in']
batch_size  = 8

train_loader, val_loader = get_dataset_and_loaders(
    csv_path, patches_dir, weather_cols, batch_size
)


In [ ]:
# Cell 5: Model + clamped Focal-Tversky + optimizer/scheduler
import segmentation_models_pytorch as smp

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = smp.Unet('resnet34', encoder_weights='imagenet',
                 in_channels=3, classes=1).to(DEVICE)

def focal_tversky_loss(logits, targets,
                       alpha=0.7, beta=0.3, gamma=0.75, eps=1e-6):
    probs = torch.sigmoid(logits)
    dims  = (2,3)
    TP = (probs * targets).sum(dim=dims)
    FN = ((1-probs) * targets).sum(dim=dims)
    FP = (probs * (1-targets)).sum(dim=dims)

    denom = TP + alpha*FN + beta*FP + eps
    denom = torch.clamp(denom, min=eps)
    tversky = (TP + eps) / denom
    return ((1 - tversky) ** gamma).mean()

optimizer = optim.Adam(model.parameters(), lr=1e-5)
scaler    = torch.cuda.amp.GradScaler()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3, verbose=True
)


<ipython-input-31-06a5abb7ace8>:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler    = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Cell 6: Training loop w/ live tqdm, grad-clipping, no NaNs
NUM_EPOCHS = 10
best_val   = float('inf')

for epoch in range(1, NUM_EPOCHS+1):
    print(f"\n=== Epoch {epoch}/{NUM_EPOCHS} ===")

    # — Training Phase —
    train_losses = []
    model.train()
    train_bar = tqdm(train_loader, desc='Train', leave=False)
    for imgs, _, masks in train_bar:
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
        masks = torch.clamp(masks, 0.0, 1.0)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            preds = model(imgs)
            if masks.shape[2:] != preds.shape[2:]:
                masks = F.interpolate(masks,
                                      size=preds.shape[2:],
                                      mode='nearest')
            loss = focal_tversky_loss(preds, masks)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        train_losses.append(loss.item())
        train_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_train = sum(train_losses) / len(train_losses)

    # — Validation Phase —
    val_losses = []
    model.eval()
    valid_bar = tqdm(val_loader, desc='Valid', leave=False)
    with torch.no_grad():
        for imgs, _, masks in valid_bar:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)
            masks = torch.clamp(masks, 0.0, 1.0)

            preds = model(imgs)
            if masks.shape[2:] != preds.shape[2:]:
                masks = F.interpolate(masks,
                                      size=preds.shape[2:],
                                      mode='nearest')
            loss = focal_tversky_loss(preds, masks)

            val_losses.append(loss.item())
            valid_bar.set_postfix(loss=f"{loss.item():.4f}")

    avg_val = sum(val_losses) / len(val_losses)
    print(f"Epoch {epoch}/{NUM_EPOCHS} → "
          f"Avg Train: {avg_train:.4f} | Avg Val: {avg_val:.4f}")

    scheduler.step(avg_val)
    if avg_val < best_val:
        best_val = avg_val
        torch.save(model.state_dict(),
                   "/content/drive/MyDrive/best_unet_resnet34_ftl.pth")
        print("🔖 Saved new best model!")

print("✅ Training complete.")



=== Epoch 1/10 ===


Train:   0%|          | 0/1429 [00:00<?, ?it/s]

<ipython-input-32-107023bb7781>:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Valid:   0%|          | 0/358 [00:00<?, ?it/s]

Epoch 1/10 → Avg Train: 0.5072 | Avg Val: 0.4445
🔖 Saved new best model!

=== Epoch 2/10 ===


...

Valid:   0%|          | 0/358 [00:00<?, ?it/s]

Epoch 9/10 → Avg Train: 0.3497 | Avg Val: 0.3532
🔖 Saved new best model!

=== Epoch 10/10 ===


Train:   0%|          | 0/1429 [00:00<?, ?it/s]

Valid:   0%|          | 0/358 [00:00<?, ?it/s]

Epoch 10/10 → Avg Train: 0.3487 | Avg Val: 0.3544
✅ Training complete.
